In [103]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

import requests
import yfinance as yf


In [2]:
sns.set()

In [3]:
df = pd.read_csv('../data.csv')

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_11647/3454644713.py:1: DtypeWarning: Columns (3,4,7,8,11,12,14,15,16,19,45,46,49,56,69,83,95,98,99,100,104,108,109,119,120,121,122,123,124,125,127,128,129,141,143,144,145,146,147,148,155,188,217,218,219,220,221,222,223,224,226,229,232,233,234,235,236,237,238,239,240,243,244,245,246,248,249,250,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


In [4]:
df = df[df['psic'].notna()] #drop 45 NaN participant SIC
telDF = df[df['psicp'].str.contains('481')] #Create df who have at least one participant in the tellcom industry with SIC: https://siccode.com/sic-code/481/telephone-communications
telDF = telDF.drop('Unnamed: 0', axis=1)
telDF = telDF.reset_index(drop=True)

#Export
telDF.to_csv('teldf.csv', index=False) #create csv sample

### Analyse companies in DF

In [ ]:
kortDF = telDF[['da', 'p', 'psicp', 'nump', 'pticker', 'sfc']]

kortDF['nump'] = kortDF['nump'].astype(int)

pattern = r'\\n' #Regex pattern to split name

#Create seperate columns for each participsnt
for n in range(1, (int(kortDF["nump"].max()) + 1)): 
	kortDF['p' + str(n) + "name"] = kortDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant

	kortDF["p" + str(n) + "sicp"] = kortDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate columns to split participant SIC codes

	kortDF["p" + str(n) + "ticker"] = kortDF['pticker'].str.split(pattern, expand=True)[n-1]#Create seperate collumns for participants identifiers

In [104]:
#Pattern to select names of individual companies
pattern = r'p\d+ticker'

#Get collumns of indivisual companies
foo = kortDF.filter(regex=pattern, axis=1)

#Create DF of companies with their occurances in the entire dataset
companies = pd.DataFrame(foo.apply(lambda x: x.value_counts())
					.sum(axis=1, skipna=True)
					.sort_values(ascending= False), columns=['alliance_count']).reset_index()

companies = companies.rename(columns={'index': 'ticker'})

#slice and keep only 100 top companies by alliance count
companies = companies.nlargest(102, columns='alliance_count')
companies = companies[companies.ticker != ''].reset_index(drop=True)

In [ ]:
companies['name'] = ''

#Get names for companies who are publically listed
def getName(ticker):
	print(ticker)
	response = yf.Ticker(ticker)

	if (response and ('longName' in response.info)):
		return response.info['longName']
	else:
		return None


def apply_complex_function(x): return getName(x['ticker'])

companies['name'] = companies.apply(apply_complex_function, axis=1)

In [135]:
companies.to_csv('companies.csv')

In [ ]:
import urllib3

fin_data = {}

#Functioon to source historic price data through yahoo Finance
def get_yfinance_data(ticker):
	print(ticker)
	
	url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1=1072915200&period2=1419984000&interval=1mo&events=history&includeAdjustedClose=true'

	headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

	if (requests.get(url, headers=headers).status_code == 200):
		data = pd.read_csv(url)
	else:
		data = None
	
	return data

#Funciton to split and arragne data to before and after discontinuity for a given company
def getPrices(ticker):
	te = get_yfinance_data(ticker)

	if (type(te) != pd.core.frame.DataFrame):
		return (None, None)

	te['Date'] = pd.to_datetime(te['Date'])
	te = te.set_index('Date', drop=True)
	p_before = (te[te.index.year <= 2009]['Close']).mean()
	p_after = (te[te.index.year <= 2009]['Close']).mean()

	return (p_before, p_after)


price_bd = []
price_ad = []

#Create two sample lists for the prices averege price before and after
for t in companies['ticker']:
	vals = getPrices(t)

	price_bd.append(vals[0])
	price_ad.append(vals[1])

In [242]:
#Add price lists to dataframe
companies['price_bd'] = price_bd
companies['price_ad'] = price_ad